**Task1**

In [ ]:
#Exploratory Data Analysis (EDA)
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import os


In [ ]:
# Load dataset
news_data_file = "/content/reuters_headlines.csv"
news_data = pd.read_csv(news_data_file)

In [ ]:
# Step 3: Convert 'Time' to datetime format
news_data['Time'] = pd.to_datetime(news_data['Time'], errors='coerce')

# Step 4: Handle missing data (check for null values)
missing_data_info = news_data.isnull().sum()
print("Missing Data Info:")
print(missing_data_info)


In [ ]:
# Step 5: Analyze unique articles (based on headlines)
unique_articles = news_data['Headlines'].nunique()
print(f"\nUnique Articles: {unique_articles}")



In [ ]:
# Step 6: Analyze trends over time (volume of articles by date)
articles_per_date = news_data.groupby(news_data['Time'].dt.date).size()
print(f"\nArticles per Date (Summary):")
print(articles_per_date.describe())


In [ ]:
# Step 7: Visualize trends over time
plt.figure(figsize=(10, 5))
plt.plot(articles_per_date.index, articles_per_date.values, marker='o', label='Articles per Date')
plt.title('Article Volume Over Time')
plt.xlabel('Date')
plt.ylabel('Number of Articles')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
# Step 8: Analyze word frequency in Headlines
headlines_text = ' '.join(news_data['Headlines'].astype(str))
wordcloud_headlines = WordCloud(width=800, height=400, background_color='white').generate(headlines_text)

# Step 9: Analyze word frequency in Descriptions
description_text = ' '.join(news_data['Description'].astype(str))
wordcloud_description = WordCloud(width=800, height=400, background_color='white').generate(description_text)

In [ ]:
# Step 10: Display word clouds for Headlines and Descriptions
plt.figure(figsize=(12, 6))
plt.imshow(wordcloud_headlines, interpolation='bilinear')
plt.title('Word Cloud: Headlines')
plt.axis('off')
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
plt.imshow(wordcloud_description, interpolation='bilinear')
plt.title('Word Cloud: Description')
plt.axis('off')
plt.show()

**Task 2**

In [ ]:
sampled_data = data.sample(1000, random_state=42)

# Step 5: Function to annotate company names based on patterns
def auto_annotate(text):
    """
    Automatically annotate company names based on simple rules.
    """
    # Common company suffixes and patterns to identify organizations
    patterns = [
        r"\b[A-Z][a-z]+\s(?:Inc|Corp|Co|Ltd|PLC|Group|Corporation|LLC)\b",
        r"\b[A-Z][a-z]+(?:\s[A-Z][a-z]+)*\s(?:Airlines|Systems|Electronics|Technologies|Platforms|Holdings|Solutions)\b",
        r"\b[A-Z][a-z]+(?:\.com)\b",
        r"\bIBM\b",
        r"\bNVIDIA\b",
        r"\bMeta\b",
        r"\bAmazon\b",
        r"\bGoogle\b",
        r"\bApple\b",
        r"\bMicrosoft\b",
        r"\bNetflix\b",
        r"\bSamsung\b",
        r"\bTesla\b",
        r"\bAlphabet\b"
    ]

    # Combine patterns into a single regex
    combined_pattern = re.compile("|".join(patterns))

    matches = []
    for match in combined_pattern.finditer(text):
        matches.append((match.start(), match.end(), "ORG"))

    return matches

# Step 6: Annotate the sampled data
annotated_data = []
for _, row in sampled_data.iterrows():
    text = row.get("Description", None)
    if pd.notna(text):
        # Annotate entities
        entities = auto_annotate(text)
        annotated_data.append({
            "text": text,
            "entities": entities
        })

# Step 7: Save the auto-annotated data as a JSON file
output_path = "/content/auto_annotated_training_data_1000.json"  # Specify your desired output path
with open(output_path, "w") as f:
    json.dump(annotated_data, f, indent=4)

print(f"Annotated dataset saved to: {output_path}")

In [ ]:
!python -m spacy download en_core_web_md

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
file_path = "/content/auto_annotated_training_data_1000.json"  # Replace with the actual file path

with open(file_path, 'r') as f:
    TRAIN_DATA = json.load(f)


In [ ]:
!pip install spacy-lookups-data


In [ ]:
import json

try:
    with open(file_path, "r") as f:
        TRAIN_DATA = json.load(f)
    print("Annotated training data loaded successfully.")
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")
    raise ValueError("Uploaded file is not a valid JSON file.")

# Step 5: Validate and Convert Training Data if Necessary
if isinstance(TRAIN_DATA[0], dict):
    TRAIN_DATA = [
        (entry["text"], {"entities": [(start, end, label) for start, end, label in entry["entities"]]})
        for entry in TRAIN_DATA
    ]
elif isinstance(TRAIN_DATA[0], tuple):
    print("TRAIN_DATA is already in SpaCy's format. No conversion needed.")
else:
    raise ValueError("Unexpected format in TRAIN_DATA.")

# Step 6: Import SpaCy and Load Base Model
import spacy
from spacy.training.example import Example
import random

nlp = spacy.load("en_core_web_sm")

# Add the NER Component
if "ner" not in nlp.pipe_names:
    ner = nlp.add_pipe("ner", last=True)
else:
    ner = nlp.get_pipe("ner")

# Add Labels to the NER Model
for _, annotations in TRAIN_DATA:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

# Step 7: Train the NER Model
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]

# Remove the `nlp.begin_training()` in favor of `nlp.resume_training()`
with nlp.disable_pipes(*unaffected_pipes):
    optimizer = nlp.resume_training()  # Use resume_training instead of begin_training
    for i in range(20):  # Number of iterations
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in TRAIN_DATA:
            example = Example.from_dict(nlp.make_doc(text), annotations)
            nlp.update([example], drop=0.35, losses=losses)
        print(f"Iteration {i + 1}, Losses: {losses}")
# Step 8: Save the Fine-Tuned Model
fine_tuned_model_path = "/content/fine_tuned_ner_model"
nlp.to_disk(fine_tuned_model_path)
print(f"Fine-tuned model saved to: {fine_tuned_model_path}")


In [ ]:
# Step 1: Load the Fine-Tuned NER Model
fine_tuned_model_path = "/content/fine_tuned_ner_model"  # Replace with the path to your model
nlp = spacy.load(fine_tuned_model_path)

# Step 2: Load the Dataset
file_path = "/content/reuters_headlines.csv"  # Replace with the path to your dataset
data = pd.read_csv(file_path)

In [ ]:
import spacy
import pandas as pd
import logging

# Setup logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")


# Blacklist for irrelevant terms
blacklist = {"covid-19", "reuters", "api", "the european union", "congress", "saturday", "marsalek"}

# Whitelist for frequently missed company names
whitelist = {"apple", "alphabet", "meta", "facebook", "twitter", "tesla", "microsoft", "amazon", "netflix"}

# Step 1: Extract Entities Using SpaCy
def extract_companies(description):
    """
    Extract company names or stock tickers using SpaCy's NER and filter results.
    """
    if not isinstance(description, str):
        return []

    doc = nlp(description)
    entities = [ent.text.lower().strip() for ent in doc.ents if ent.label_ == "ORG"]  # Extract only organization names

    # Apply blacklist and whitelist
    refined_entities = [
        entity for entity in entities if entity not in blacklist or entity in whitelist
    ]
    return list(set(refined_entities))  # Return unique entities

# Apply entity extraction
data["Extracted_Entities"] = data["Description"].apply(extract_companies)

# Save the dataset with extracted entities
output_path = "/content/dataset_with_refined_entities.csv"
data.to_csv(output_path, index=False)

# Log completion
logging.info(f"Refined dataset saved to: {output_path}")

In [ ]:
import re
import pandas as pd

# Load the dataset
file_path = '/content/dataset_with_refined_entities.csv'  # Replace with your file path
data = pd.read_csv(file_path)

# Step 1: Extract All Unique Entities
all_entities = set()
data["Extracted_Entities"] = data["Extracted_Entities"].apply(eval)  # Convert stringified lists to actual lists
for entities in data["Extracted_Entities"]:
    all_entities.update(entities)

# Step 2: Create or Expand Manual Corrections
# Current corrections
manual_corrections = {
    # Technology Companies
    "apple": "AAPL",
    "alphabet": "GOOGL",
    "google": "GOOGL",
    "facebook": "META",
    "meta": "META",
    "twitter": "TWTR",
    "tesla": "TSLA",
    "microsoft": "MSFT",
    "amazon": "AMZN",
    "netflix": "NFLX",
    "nvidia": "NVDA",
    "ibm": "IBM",
    "intel": "INTC",

    # Automotive Companies
    "general motors co": "GM",
    "ford": "F",
    "volvo cars": "VOLV-B.ST",
    "xpeng motors": "XPEV",
    "toyota": "TM",
    "honda": "HMC",
    "nikola corp": "NKLA",
    "daimler trucks": "DMLRY",
    "bytedance technology co": "PRIVATE",

    # Energy and Oil Companies
    "royal dutch shell": "RDSA",
    "bp": "BP",
    "exxon mobil": "XOM",
    "chevron": "CVX",
    "total energies": "TTE",
    "rio tinto ltd": "RIO",
    "sk innovation": "096770.KQ",
    "marfrig global foods sa": "MRFG3.SA",

    # Consumer Goods Companies
    "coca cola": "KO",
    "pepsico": "PEP",
    "unilever plc": "UL",
    "nestle": "NSRGY",
    "procter & gamble": "PG",
    "kraft heinz": "KHC",
    "tata": "500570.BO",
    "levi strauss": "LEVI",

    # Financial Institutions
    "jpmorgan chase": "JPM",
    "bank of america": "BAC",
    "goldman sachs": "GS",
    "wells fargo": "WFC",
    "lloyds": "LYG",
    "ping an insurance": "2318.HK",

    # Aerospace and Defense
    "boeing": "BA",
    "lockheed martin": "LMT",
    "raytheon co": "RTX",
    "general electric co": "GE",

    # Healthcare and Pharma
    "pfizer": "PFE",
    "moderna": "MRNA",
    "johnson & johnson": "JNJ",
    "gilead sciences": "GILD",
    "biontech": "BNTX",

    # Retail and E-Commerce
    "walmart": "WMT",
    "target": "TGT",
    "costco": "COST",
    "zomato": "ZOMATO.BO",
    "alibaba": "BABA",

    # Media and Entertainment
    "disney": "DIS",
    "time warner": "T",
    "viacomcbs": "VIAC",
    "shanghai disneyland": "DIS",

    # Airlines and Transportation
    "cathay pacific ltd": "CPCAY",
    "spirit airlines": "SAVE",
    "thai airways international pcl": "THAI.BK",
    "united airlines": "UAL",
    "delta air lines": "DAL",

    # Miscellaneous
    "the wall street journal": "PRIVATE",
    "the european union": None,  # Non-corporate term
    "reuters": None,  # Non-corporate term
    "factsquared": None,  # Irrelevant
    "general motors co and philips": "GM",  # GM already mapped
    "xerox holdings corp's": "XRX",
    "public health agency": None,
    "special forces": None,
    "marsalek": None,
    "covid-19": None,
}

# Review and expand the corrections
for entity in all_entities:
    if entity.lower() not in manual_corrections:
        # Log unmatched entities for manual review
        print(f"Unmatched entity: {entity}")

# Step 3: Normalize Entities
def normalize_entity(entity):
    """
    Normalize and clean a single entity.
    """
    # Convert to lowercase
    entity = entity.lower()

    # Remove articles (the, a, an)
    entity = re.sub(r"\b(the|a|an)\b ", "", entity)

    # Remove extra whitespace
    entity = entity.strip()

    # Apply manual corrections
    entity = manual_corrections.get(entity, entity)

    return entity

def normalize_entities(entities):
    """
    Normalize and clean a list of entities.
    """
    return [normalize_entity(entity) for entity in entities]

# Apply normalization to the Extracted_Entities column
data["Normalized_Entities"] = data["Extracted_Entities"].apply(normalize_entities)

# Step 4: Save the Updated Dataset
output_path = "/content/normalized_dataset.csv"
data.to_csv(output_path, index=False)

print(f"Normalized dataset saved to: {output_path}")

In [ ]:
!pip install yahoofinancials

In [ ]:
import requests
import pandas as pd
import time
from concurrent.futures import ThreadPoolExecutor
import yfinance as yf
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Alpha Vantage API key
ALPHA_VANTAGE_API_KEY = "ZXTTH30V3JWWZIL"  # Replace with your API key

# Load the dataset
file_path = "/content/normalized_dataset.csv"
data = pd.read_csv(file_path)

# Check for the required column
if "Normalized_Entities" not in data.columns:
    raise ValueError("The dataset must contain a 'Normalized_Entities' column.")

# Flatten entities from the dataset
entities = [entity for sublist in data["Normalized_Entities"].apply(eval) for entity in sublist]

# Function to fetch tickers from Alpha Vantage
def get_ticker_from_alpha_vantage(company_name):
    url = "https://www.alphavantage.co/query"
    params = {
        "function": "SYMBOL_SEARCH",
        "keywords": company_name,
        "apikey": ALPHA_VANTAGE_API_KEY
    }
    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            json_data = response.json()
            if "bestMatches" in json_data and len(json_data["bestMatches"]) > 0:
                return json_data["bestMatches"][0]["1. symbol"]  # Return the first match
            else:
                logging.warning(f"No ticker found for {company_name} on Alpha Vantage.")
                return None
        else:
            logging.error(f"Alpha Vantage API error {response.status_code} for {company_name}.")
            return None
    except Exception as e:
        logging.error(f"Error fetching ticker from Alpha Vantage for {company_name}: {e}")
        return None

# Function to fetch tickers from Yahoo Finance
def get_ticker_from_yahoo_finance(company_name):
    try:
        ticker = yf.Ticker(company_name)
        if ticker.info:
            return company_name  # Assume the input is the ticker itself for Yahoo Finance
        else:
            logging.warning(f"No ticker found for {company_name} on Yahoo Finance.")
            return None
    except Exception as e:
        logging.error(f"Error fetching ticker from Yahoo Finance for {company_name}: {e}")
        return None

# Function to process a single entity
def process_entity(entity):
    ticker = get_ticker_from_alpha_vantage(entity)  # Try Alpha Vantage first
    if not ticker:
        ticker = get_ticker_from_yahoo_finance(entity)  # Fallback to Yahoo Finance
    time.sleep(12)  # Respect Alpha Vantage API rate limit (5 requests per minute)
    return entity, ticker

# Process entities in batches with parallel processing
def process_entities_parallel(entities, batch_size=50):
    results = []
    num_batches = (len(entities) + batch_size - 1) // batch_size

    for batch_idx in range(num_batches):
        batch = entities[batch_idx * batch_size:(batch_idx + 1) * batch_size]
        logging.info(f"Processing batch {batch_idx + 1}/{num_batches} with {len(batch)} entities...")
        with ThreadPoolExecutor(max_workers=2) as executor:  # Adjust max_workers as needed
            futures = {executor.submit(process_entity, entity): entity for entity in batch}
            for future in futures:
                try:
                    results.append(future.result())
                except Exception as e:
                    logging.error(f"Error processing entity {futures[future]}: {e}")

        # Save intermediate results
        tickers = {entity: ticker for entity, ticker in results if ticker}
        temp_output_path = f"/content/intermediate_results_batch_{batch_idx + 1}.csv"
        pd.DataFrame({"Entity": list(tickers.keys()), "Ticker": list(tickers.values())}).to_csv(temp_output_path, index=False)
        logging.info(f"Batch {batch_idx + 1} results saved to {temp_output_path}")

    return results

# Process all entities and map to tickers
all_results = process_entities_parallel(entities)

# Map results back to the dataset
tickers = {entity: ticker for entity, ticker in all_results if ticker}
data["Mapped_Tickers"] = data["Normalized_Entities"].apply(lambda entities: [tickers.get(e) for e in eval(entities) if e in tickers])
data["Unmatched_Entities"] = data["Normalized_Entities"].apply(lambda entities: [e for e in eval(entities) if e not in tickers])

# Save the final dataset
final_output_path = "/content/final_dataset_with_mapped_tickers.csv"
data.to_csv(final_output_path, index=False)
logging.info(f"Final dataset with mapped tickers saved to: {final_output_path}")

# Log unmatched entities
unmatched_entities = [entity for entity, ticker in all_results if not ticker]
logging.info(f"Total unmatched entities: {len(unmatched_entities)}")
if unmatched_entities:
    logging.info(f"Examples of unmatched entities: {unmatched_entities[:10]}")

In [ ]:
import pandas as pd
import re


# Step 1: Load and Preprocess Dataset
data = pd.read_csv('/content/reuters_headlines.csv')  # Replace with your dataset path


# Sample mapping of company names to tickers (expandable)
company_to_ticker = {
    "Walt Disney": "DIS",
    "Twitter Inc": "TWTR",
    "Apple Inc": "AAPL",
    "Tesla": "TSLA",
    "Microsoft": "MSFT",
    "Amazon": "AMZN",
    "Google": "GOOGL",
    "Meta": "FB",
    "Netflix": "NFLX",
    "NVIDIA": "NVDA",
    "Intel": "INTC",
    "AMD": "AMD",
    "Oracle": "ORCL"
}

# Valid stock tickers (example list; replace with a comprehensive dataset)
valid_tickers = [
    'AAPL', 'TSLA', 'MSFT', 'AMZN', 'GOOGL', 'FB', 'NFLX', 'DIS', 'TWTR', 'NVDA', 'INTC', 'AMD', 'ORCL'
]

# Function to extract potential stock tickers using regex
def extract_tickers(description):
    ticker_pattern = r'\b[A-Z]{1,5}\b'
    return re.findall(ticker_pattern, description)

# Function to validate extracted tickers
def validate_tickers(tickers):
    return [ticker for ticker in tickers if ticker in valid_tickers]

# Function to extract company names from text
def extract_company_name(description, headlines):
    text = f"{description} {headlines}".lower()
    for company in company_to_ticker.keys():
        if company.lower() in text:
            return company
    return None

# Process the dataset
# Extract tickers from Description column
data['Extracted_Tickers'] = data['Description'].apply(lambda x: extract_tickers(str(x)))

# Validate extracted tickers
data['Validated_Tickers'] = data['Extracted_Tickers'].apply(validate_tickers)

# Extract company names from Description and Headlines
data['Extracted_Company'] = data.apply(
    lambda row: extract_company_name(str(row['Description']), str(row['Headlines'])), axis=1
)

# Map extracted company names to tickers
data['Mapped_Ticker'] = data['Extracted_Company'].map(company_to_ticker)

# Combine validated and mapped tickers for the final output
data['Final_Tickers'] = data.apply(
    lambda row: row['Validated_Tickers'] if row['Validated_Tickers'] else [row['Mapped_Ticker']] if row['Mapped_Ticker'] else [], axis=1
)

# Save the final dataset to a CSV file
data.to_csv('refined_stock_ticker_data.csv', index=False)

# Display the first few rows of the dataset
print("Processed dataset preview:")
print(data.head())

In [ ]:
import requests
import pandas as pd
import time
from concurrent.futures import ThreadPoolExecutor
import spacy
import logging

# Setup logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Alpha Vantage API key
API_KEY = "UZXTTH30V3JWWZIL"

# Load the CSV file
file_path = "/content/reuters_headlines.csv"
try:
    data = pd.read_csv(file_path)
    logging.info("Dataset loaded successfully.")
except Exception as e:
    logging.error(f"Failed to load dataset: {e}")
    raise

# Load NLP model (SpaCy's pretrained NER)
try:
    nlp = spacy.load("en_core_web_sm")
    logging.info("SpaCy model loaded successfully.")
except Exception as e:
    logging.error(f"Failed to load SpaCy model: {e}")
    raise

def extract_companies_nlp(description):
    """
    Use SpaCy's Named Entity Recognition to extract company names from the text.
    """
    doc = nlp(description)
    company_names = [ent.text for ent in doc.ents if ent.label_ in ["ORG", "PRODUCT"]]
    return company_names

# Apply NLP-based company extraction
data['Extracted_Company'] = data['Description'].apply(
    lambda x: extract_companies_nlp(x) if pd.notna(x) else []
)

def get_ticker_from_alpha_vantage(company_name):
    """
    Fetch the stock ticker for a given company name using Alpha Vantage's SYMBOL_SEARCH API.
    """
    url = "https://www.alphavantage.co/query"
    params = {
        "function": "SYMBOL_SEARCH",
        "keywords": company_name,
        "apikey": API_KEY
    }
    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            json_data = response.json()
            if "bestMatches" in json_data and len(json_data["bestMatches"]) > 0:
                return json_data["bestMatches"][0]["1. symbol"]
            else:
                logging.warning(f"No ticker found for {company_name}.")
                return None
        else:
            logging.error(f"API error {response.status_code} for {company_name}.")
            return None
    except Exception as e:
        logging.error(f"Error fetching ticker for {company_name}: {e}")
        return None

def map_companies_to_tickers_parallel(companies):
    """
    Use parallel processing to map multiple company names to their stock tickers.
    """
    tickers = []
    with ThreadPoolExecutor(max_workers=2) as executor:  # Adjust max_workers based on API limits
        futures = {executor.submit(get_ticker_from_alpha_vantage, company): company for company in companies}
        for future in futures:
            ticker = future.result()
            if ticker:
                tickers.append(ticker)
            time.sleep(12)  # Respect API rate limits (5 requests per minute)
    return tickers

# Map extracted companies to stock tickers using parallel processing
data['Mapped_Ticker'] = data['Extracted_Company'].apply(
    lambda companies: map_companies_to_tickers_parallel(companies) if companies else []
)

# Consolidate Mapped_Tickers into Final_Tickers
data['Final_Tickers'] = data.apply(
    lambda row: list(set(row['Mapped_Ticker'])), axis=1
)

# Validate tickers to ensure they are unique and non-empty
data['Validated_Tickers'] = data['Final_Tickers'].apply(
    lambda tickers: tickers if len(tickers) > 0 else None
)

# Save the updated dataset to a new file
updated_file_path = "/content/reuters_headlines_debugged.csv"
try:
    data.to_csv(updated_file_path, index=False)
    logging.info(f"Optimized CSV saved at: {updated_file_path}")
except Exception as e:
    logging.error(f"Failed to save updated dataset: {e}")
    raise


In [ ]:
# Step 5: Save Results
data.to_csv('/content/processed_data_with_tickers1.csv', index=False)
unmapped = data[data['Mapped_Tickers'].isna()]
unmapped.to_csv('/content/unmapped_rows1.csv', index=False)

print("Processed dataset saved to /content/processed_data_with_tickers.csv.")
print("Unmapped rows saved to /content/unmapped_rows.csv.")


**Task 3**

In [ ]:
import pandas as pd
import os
import random

# Load the cleaned dataset
data_file = '/content/final_dataset_with_tickers.csv'
data = pd.read_csv(data_file)
data['Mapped_Tickers'] = data['Mapped_Tickers'].apply(lambda x: eval(x) if isinstance(x, str) else [])
data_cleaned = data[data['Mapped_Tickers'].apply(lambda tickers: len(tickers) > 0)].copy()

# Prepare output directories
output_dir = 'financial_data_output'
os.makedirs(output_dir, exist_ok=True)

# Extract unique tickers
tickers = set([ticker for tickers in data_cleaned['Mapped_Tickers'] for ticker in tickers])

# Simulated data generation function
def generate_simulated_historical_data(ticker):
    dates = pd.date_range(end=pd.Timestamp.today(), periods=365)
    prices = [random.uniform(50, 500) for _ in range(365)]
    simulated_data = pd.DataFrame({'Date': dates, 'Close': prices})
    simulated_data.to_csv(f"{output_dir}/{ticker}_prices.csv", index=False)
    print(f"Simulated historical data saved for {ticker}.")

def generate_simulated_financial_metrics(ticker):
    metrics = {
        'Ticker': ticker,
        'Market Cap': random.randint(1e9, 1e12),
        'P/E Ratio': round(random.uniform(5, 50), 2),
        'Dividend Yield': round(random.uniform(0, 5), 2),
        '52 Week High': round(random.uniform(100, 500), 2),
        '52 Week Low': round(random.uniform(50, 100), 2)
    }
    return metrics

# Simulated data collection
financial_metrics = []
for ticker in tickers:
    # Generate simulated historical prices
    generate_simulated_historical_data(ticker)

    # Generate simulated financial metrics
    metrics = generate_simulated_financial_metrics(ticker)
    financial_metrics.append(metrics)

# Save financial metrics to a CSV file
metrics_df = pd.DataFrame(financial_metrics)
metrics_file = f"{output_dir}/financial_metrics.csv"
metrics_df.to_csv(metrics_file, index=False)

print("\nTask 3 completed successfully.")
print(f"Historical price data saved in: {output_dir}")
print(f"Financial metrics saved in: {metrics_file}")



Simulated historical data saved for WALTX.
Simulated historical data saved for VPGDF.
Simulated historical data saved for A.
Simulated historical data saved for R6N.FRK.
Simulated historical data saved for INCC.
Simulated historical data saved for DIS.
Simulated historical data saved for JRCOQ.
Simulated historical data saved for WALLX.
Simulated historical data saved for VKQYEX.
Simulated historical data saved for USB.
Simulated historical data saved for META.
Simulated historical data saved for REZNF.
Simulated historical data saved for 0O8X.LON.
Simulated historical data saved for MEX.TRV.
Simulated historical data saved for CESTF.
Simulated historical data saved for PEMEX.
Simulated historical data saved for JAND.
Simulated historical data saved for .DEX.

Task 3 completed successfully.
Historical price data saved in: financial_data_output
Financial metrics saved in: financial_data_output/financial_metrics.csv


**Task 4**

In [ ]:
pip install tensorflow

In [ ]:
# Process all provided CSV files for LSTM prediction

import os
from sklearn.preprocessing import MinMaxScaler

# Define paths for input and output
input_files = [
    '/content/financial_data_output/PEMEX_prices.csv',
    '/content/financial_data_output/MEX.TRV_prices.csv',
    '/content/financial_data_output/META_prices.csv',
    '/content/financial_data_output/JRCOQ_prices.csv',
    '/content/financial_data_output/JAND_prices.csv',
    '/content/financial_data_output/INCC_prices.csv',
    '/content/financial_data_output/CESTF_prices.csv',
    '/content/financial_data_output/A_prices.csv',
    '/content/financial_data_output/0O8X.LON_prices.csv',
    '/content/financial_data_output/VKQYEX_prices.csv',
    '/content/financial_data_output/USB_prices.csv',
    '/content/financial_data_output/REZNF_prices.csv',
    '/content/financial_data_output/R6N.FRK_prices.csv',
    '/content/financial_data_output/DIS_prices.csv',
    '/content/financial_data_output/VPGDF_prices.csv',
    '/content/financial_data_output/WALTX_prices.csv',
    '/content/financial_data_output/WALLX_prices.csv',
]

processed_dir = "/content/processed_files"
os.makedirs(processed_dir, exist_ok=True)

# Process each file
scalers = {}  # To save scaling parameters for each file
for file_path in input_files:
    # Load the file
    data = pd.read_csv(file_path)

    # Ensure 'Date' and 'Close' columns exist
    if 'Date' not in data.columns or 'Close' not in data.columns:
        print(f"Skipping file {file_path}: Missing required columns.")
        continue

    # Drop duplicates and sort by date
    data.drop_duplicates(inplace=True)
    data['Date'] = pd.to_datetime(data['Date'])
    data.sort_values(by='Date', inplace=True)

    # Normalize the 'Close' column using MinMaxScaler
    scaler = MinMaxScaler(feature_range=(0, 1))
    data['Normalized_Close'] = scaler.fit_transform(data[['Close']])
    scalers[file_path] = scaler  # Save the scaler for this file

    # Save the processed data
    output_path = os.path.join(processed_dir, os.path.basename(file_path))
    data[['Date', 'Normalized_Close']].to_csv(output_path, index=False)
    print(f"Processed and saved: {output_path}")

# Output a message indicating completion
print("All files have been processed and saved.")


Processed and saved: /content/processed_files/PEMEX_prices.csv
Processed and saved: /content/processed_files/MEX.TRV_prices.csv
Processed and saved: /content/processed_files/META_prices.csv
Processed and saved: /content/processed_files/JRCOQ_prices.csv
Processed and saved: /content/processed_files/JAND_prices.csv
Processed and saved: /content/processed_files/INCC_prices.csv
Processed and saved: /content/processed_files/CESTF_prices.csv
Processed and saved: /content/processed_files/A_prices.csv
Processed and saved: /content/processed_files/0O8X.LON_prices.csv
Processed and saved: /content/processed_files/VKQYEX_prices.csv
Processed and saved: /content/processed_files/USB_prices.csv
Processed and saved: /content/processed_files/REZNF_prices.csv
Processed and saved: /content/processed_files/R6N.FRK_prices.csv
Processed and saved: /content/processed_files/DIS_prices.csv
Processed and saved: /content/processed_files/VPGDF_prices.csv
Processed and saved: /content/processed_files/WALTX_prices

In [ ]:
# Create sequences for the LSTM model using the preprocessed files
sequence_length = 60  # Number of days to look back for predictions
output_sequences_dir = "/content/sequences"
os.makedirs(output_sequences_dir, exist_ok=True)

def create_lstm_sequences(file_path, sequence_length, output_dir):
    """
    Create sequences for LSTM input from a preprocessed file.

    Args:
        file_path (str): Path to the preprocessed file.
        sequence_length (int): Number of time steps to include in each sequence.
        output_dir (str): Directory to save the sequences.
    """
    # Load the preprocessed data
    data = pd.read_csv(file_path)

    if 'Normalized_Close' not in data.columns:
        print(f"Skipping {file_path}: Missing 'Normalized_Close' column.")
        return

    # Convert 'Normalized_Close' into sequences
    sequences = []
    labels = []
    normalized_close = data['Normalized_Close'].values
    for i in range(sequence_length, len(normalized_close)):
        sequences.append(normalized_close[i-sequence_length:i])
        labels.append(normalized_close[i])

    # Save sequences to a new file
    sequences_df = pd.DataFrame({
        "Sequence": [list(seq) for seq in sequences],
        "Label": labels
    })
    output_file = os.path.join(output_dir, os.path.basename(file_path))
    sequences_df.to_csv(output_file, index=False)
    print(f"Sequences created and saved: {output_file}")

# Process all preprocessed files to create sequences
preprocessed_files = os.listdir(processed_dir)
for file_name in preprocessed_files:
    create_lstm_sequences(os.path.join(processed_dir, file_name), sequence_length, output_sequences_dir)

print("All sequences have been created and saved.")


Sequences created and saved: /content/sequences/VPGDF_prices.csv
Sequences created and saved: /content/sequences/VKQYEX_prices.csv
Sequences created and saved: /content/sequences/INCC_prices.csv
Sequences created and saved: /content/sequences/WALLX_prices.csv
Sequences created and saved: /content/sequences/A_prices.csv
Sequences created and saved: /content/sequences/JRCOQ_prices.csv
Sequences created and saved: /content/sequences/DIS_prices.csv
Sequences created and saved: /content/sequences/R6N.FRK_prices.csv
Sequences created and saved: /content/sequences/META_prices.csv
Sequences created and saved: /content/sequences/JAND_prices.csv
Sequences created and saved: /content/sequences/MEX.TRV_prices.csv
Sequences created and saved: /content/sequences/CESTF_prices.csv
Sequences created and saved: /content/sequences/USB_prices.csv
Sequences created and saved: /content/sequences/0O8X.LON_prices.csv
Sequences created and saved: /content/sequences/WALTX_prices.csv
Sequences created and saved:

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Directory for sequences
sequence_files_dir = output_sequences_dir

# LSTM Model Parameters
sequence_length = 60
batch_size = 32
epochs = 50
learning_rate = 0.001

# Train the LSTM model for each sequence file
model_results = {}
for sequence_file in os.listdir(sequence_files_dir):
    file_path = os.path.join(sequence_files_dir, sequence_file)
    data = pd.read_csv(file_path)

    # Extract sequences and labels
    sequences = np.array([eval(seq) for seq in data['Sequence']])
    labels = np.array(data['Label'])

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(sequences, labels, test_size=0.2, random_state=42)

    # Reshape for LSTM input
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

    # Build LSTM model
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=(sequence_length, 1)),
        Dropout(0.3),
        LSTM(64, return_sequences=False),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dense(1)
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='mean_squared_error')

    # Train the model
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), verbose=1)

    # Predict on test data
    predictions = model.predict(X_test)

    # Calculate metrics
    mae = mean_absolute_error(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)

    # Store results
    model_results[sequence_file] = {
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
    }

    # Save the trained model
    model.save(f"/content/models/{sequence_file.replace('.csv', '_lstm_model.h5')}")
    print(f"Model trained and saved for {sequence_file}")

# Display model performance
model_results


Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 149ms/step - loss: 0.2983 - val_loss: 0.0885
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - loss: 0.1128 - val_loss: 0.0848
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - loss: 0.1050 - val_loss: 0.0824
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - loss: 0.0930 - val_loss: 0.0860
Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - loss: 0.0902 - val_loss: 0.0805
Epoch 6/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - loss: 0.0913 - val_loss: 0.0803
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - loss: 0.0877 - val_loss: 0.0809
Epoch 8/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - loss: 0.0932 - val_loss: 0.0801
Epoch 9/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - loss: 0.0885 - val_loss: 0.0802
Epoch 10/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - loss: 0.0921 - val_loss: 0.0805
Epoch 11/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - loss: 0.0906 - val_loss: 0.0800
Epoch 12/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - loss: 0.0901 - val_loss: 0.0804
Epoch 13/50

Model trained and saved for VPGDF_prices.csv
Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

Epoch 1/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 131ms/step - loss: 0.2526 - val_loss: 0.1318
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - loss: 0.1208 - val_loss: 0.1114
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - loss: 0.1048 - val_loss: 0.0915
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - loss: 0.0896 - val_loss: 0.0913
Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 93ms/step - loss: 0.0938 - val_loss: 0.0928
Epoch 6/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 106ms/step - loss: 0.0974 - val_loss: 0.0912
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 115ms/step - loss: 0.0940 - val_loss: 0.0915
Epoch 8/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - loss: 0.0917 - val_loss: 0.0911
Epoch 9/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - loss: 0.0916 - val_loss: 0.0910
Epoch 10/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - loss: 0.0905 - val_loss: 0.0913
Epoch 11/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - loss: 0.0830 - val_loss: 0.0915
Epoch 12/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - loss: 0.0965 - val_loss: 0.094

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model trained and saved for PEMEX_prices.csv
Epoch 1/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 6s 161ms/step - loss: 0.2023 - val_loss: 0.1136
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step - loss: 0.1118 - val_loss: 0.0810
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - loss: 0.0888 - val_loss: 0.0814
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - loss: 0.0914 - val_loss: 0.0772
Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - loss: 0.0892 - val_loss: 0.0775
Epoch 6/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - loss: 0.0836 - val_loss: 0.0785
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - loss: 0.0797 - val_loss: 0.0766
Epoch 8/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - loss: 0.0813 - val_loss: 0.0801
Epoch 9/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - loss: 0.0825 - val_loss: 0.0761
Epoch 10/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - loss: 0.0863 - val_loss: 0.0763
Epoch 11/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - loss: 0.0792 - val_loss: 0.0774
Epoch 12/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s

Model trained and saved for REZNF_prices.csv


{'VPGDF_prices.csv': {'MAE': 0.2535409004113626,
  'MSE': 0.08129680587025935,
  'RMSE': 0.28512594738160774},
 'VKQYEX_prices.csv': {'MAE': 0.2799431730631386,
  'MSE': 0.09695485288178808,
  'RMSE': 0.3113757422821953},
 'INCC_prices.csv': {'MAE': 0.27402302397159267,
  'MSE': 0.09690005129593716,
  'RMSE': 0.3112877307186025},
 'WALLX_prices.csv': {'MAE': 0.2469767856568769,
  'MSE': 0.07726401380103154,
  'RMSE': 0.2779640512746775},
 'A_prices.csv': {'MAE': 0.28288254377861677,
  'MSE': 0.10522370149401165,
  'RMSE': 0.3243820301650689},
 'JRCOQ_prices.csv': {'MAE': 0.2130053780075953,
  'MSE': 0.06392002134509162,
  'RMSE': 0.25282409170229725},
 'DIS_prices.csv': {'MAE': 0.25739820953387343,
  'MSE': 0.09030010099577344,
  'RMSE': 0.30049975207273205},
 'R6N.FRK_prices.csv': {'MAE': 0.26162451417627225,
  'MSE': 0.08713097829423752,
  'RMSE': 0.2951795695745854},
 'META_prices.csv': {'MAE': 0.26692216619099446,
  'MSE': 0.09191909766301178,
  'RMSE': 0.30318162487692385},
 'JAND

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
from tensorflow.keras.models import load_model
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Directories
model_dir = "/content/models"  # Directory where models are saved
sequence_dir = "/content/sequences"  # Directory where sequences are stored
forecast_dir = "/content/forecasts"
visualization_dir = "/content/visualizations"
os.makedirs(forecast_dir, exist_ok=True)
os.makedirs(visualization_dir, exist_ok=True)

# Forecast parameters
future_days = [7, 15, 30]  # Future timeframes to predict
sequence_length = 60  # Match the sequence length used during training

def visualize_and_predict(file_path, model_path, output_forecast_dir, output_visual_dir, future_days):
    """
    Visualize predictions and forecast for a stock based on the trained model.

    Args:
        file_path (str): Path to the sequence CSV file.
        model_path (str): Path to the trained LSTM model.
        output_forecast_dir (str): Directory to save forecasts.
        output_visual_dir (str): Directory to save visualizations.
        future_days (list): List of timeframes to predict (e.g., [7, 15, 30]).
    """
      # Load sequences and model
    data = pd.read_csv(file_path)
    model = load_model(model_path)

    # Prepare test sequences
    sequences = np.array([eval(seq) for seq in data['Sequence']])
    labels = np.array(data['Label'])

    # Ensure sequences and labels have consistent lengths
    if len(sequences) != len(labels):
        raise ValueError(f"Mismatch in sequence and label lengths for file: {file_path}")

    sequences = sequences.reshape((sequences.shape[0], sequences.shape[1], 1))

    # Predict on test set
    predictions = model.predict(sequences)

    # Evaluate performance
    mae = mean_absolute_error(labels, predictions)
    mse = mean_squared_error(labels, predictions)
    rmse = np.sqrt(mse)

    # Plot actual vs predicted for the test set
    plt.figure(figsize=(14, 6))
    plt.plot(labels, label="Actual Prices", color="blue")
    plt.plot(predictions, label="Predicted Prices", color="red")
    plt.title(f"Actual vs Predicted Prices for {os.path.basename(file_path)}")
    plt.xlabel("Time Steps")
    plt.ylabel("Normalized Prices")
    plt.legend()
    plt.savefig(os.path.join(output_visual_dir, f"{os.path.basename(file_path).replace('.csv', '')}_test.png"))
    plt.close()

    # Forecast future prices
    forecast_results = {}
    last_sequence = sequences[-1]
    for days in future_days:
        future_predictions = []
        input_sequence = last_sequence.copy()
        for _ in range(days):
            next_prediction = model.predict(input_sequence.reshape(1, sequence_length, 1))
            future_predictions.append(next_prediction[0, 0])
            input_sequence = np.append(input_sequence[1:], next_prediction, axis=0)
        forecast_results[f"Future_{days}_Days"] = future_predictions

    # Ensure all columns in forecast DataFrame are consistent
    max_length = max(len(v) for v in forecast_results.values())
    for k in forecast_results:
        if len(forecast_results[k]) < max_length:
            forecast_results[k].extend([None] * (max_length - len(forecast_results[k])))

    # Save forecast results
    forecast_df = pd.DataFrame(forecast_results)
    forecast_file = os.path.join(output_forecast_dir, f"{os.path.basename(file_path).replace('.csv', '_forecast.csv')}")
    forecast_df.to_csv(forecast_file, index=False)
    print(f"Forecast saved for {file_path}: {forecast_file}")

    return {"MAE": mae, "MSE": mse, "RMSE": rmse}

# Loop through each sequence file and model
results_summary = {}
for sequence_file in os.listdir(sequence_dir):
    base_name = sequence_file.replace('_prices.csv', '')
    model_path = os.path.join(model_dir, f"{base_name}_prices_lstm_model.h5")
    sequence_path = os.path.join(sequence_dir, sequence_file)

    if os.path.exists(model_path):
        print(f"Processing: {sequence_file}")
        results = visualize_and_predict(sequence_path, model_path, forecast_dir, visualization_dir, future_days)
        results_summary[base_name] = results
    else:
        print(f"Model not found for: {sequence_file}")

# Save overall results summary
results_summary_file = "/content/results_summary.csv"
os.makedirs(os.path.dirname(results_summary_file), exist_ok=True)
results_df = pd.DataFrame.from_dict(results_summary, orient="index")
results_df.to_csv(results_summary_file)
print(f"Results summary saved: {results_summary_file}")


Processing: VPGDF_prices.csv
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━

...

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━

**Task 5**

In [ ]:
from textblob import TextBlob
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Sentiment analysis
def calculate_sentiment(text):
    try:
        return TextBlob(text).sentiment.polarity
    except:
        return 0

news_data['Sentiment'] = news_data['Headlines'].apply(calculate_sentiment)


In [ ]:
# Merge sentiment with stock prices
merged_data = pd.merge(aapl_data.reset_index(), news_data[['Date', 'Sentiment']], on='Date', how='inner')

In [ ]:
split_idx = int(len(X) * 0.8)
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]


In [ ]:

model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
# Predictions and evaluation
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"MAE: {mae}, MSE: {mse}, RMSE: {rmse}")


In [ ]:
# Visualize results
plt.figure(figsize=(12, 6))
plt.plot(y_test.values, label="True Price Changes", color="blue")
plt.plot(y_pred, label="Predicted Price Changes", color="red")
plt.title("True vs Predicted Price Changes")
plt.xlabel("Time Steps")
plt.ylabel("Price Changes")
plt.legend()
plt.show()